# **Establishing a Connection Between Google Drive and Google Colab**

To streamline our workflow and seamlessly access project-related files, we'll establish a connection with Google Drive. This integration ensures a smooth collaboration between Google Drive and Google Colab, enabling efficient handling of relevant files throughout the project.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Install necessary libraries**
To kickstart our anomaly detection tasks, it's crucial to acquire key libraries that bring essential functionalities to our workflow:

1.OpenL3 - Audio Embedding Framework

Purpose: OpenL3 is a powerful codebase designed for audio embedding, a crucial step in our processing pipeline. It excels in extracting high-level representations from audio signals.

Features: OpenL3 provides robust feature extraction, capturing salient characteristics like timbre and rhythm. Its versatility makes it suitable for applications ranging from music analysis to sound event recognition.

2.Librosa - Audio Signal Processing Library

Purpose: Librosa is a comprehensive library tailored for audio signal processing, offering tools for analysis and manipulation of audio signals.

Features: Librosa supports spectral analysis, feature extraction, and visualization. It is widely used in music information retrieval and machine learning, enabling tasks such as generating spectrograms and extracting tempo and beat information.

By incorporating OpenL3 and Librosa into our toolkit, we enhance our audio processing capabilities, extracting meaningful features and insights with precision and efficiency.

In [2]:
!pip install openl3
!pip install librosa

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openl3: filename=openl3-0.4.2-py2.py3-none-any.whl size=249327032 sha256=94e6489d7fc3c24eac4acc42880389abbc0c13838ee0ea84eb7ef411db9687b1
  Stored in directory: /root/.cache/pip/wheels/d0/4d/0a/e57b1dc8ead91b3c5709d9de4f02d1cdd3a91f609a8f1c1062
  Created wheel for kapre: filename=kapre-0.3.7-py3-none-any.whl size=29604 sha256=7eb638de3f3ae54c42b2b17d95c76eec102b95f94d81e575ac74b159f09af77c
  Stored in directory: /root/.cache/pip/wheels/3e/54/f9/37e9f36590a5431dc207f7dc0eb2e607ff8958d5728e45ef58
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320708 sha256=9bba8a5e5c989ad625a6839058f3f568be50687c7319c595c4bf8b3039be5a02
  Stored in directory: /root

# **Import relevant libraries**
Let's transition to the next phase by importing pertinent libraries essential for our tasks. These libraries will facilitate operations related to the operating system, file handling, audio file manipulation, audio embedding, and classification tasks.

In [3]:
# Import relevant libraries to work with files and with the operating system
import zipfile
import os

# Import relevant libraries for working with audio files as well as audio
# embedding
import librosa
import openl3

# Import relevant libraries for performing calculations and classification
import numpy as np
from sklearn.svm import SVC


# **Extracting Data: Unzipping Audio Samples for Processing**
Our next step involves unpacking a zip file containing the crucial audio samples essential for our work. By extracting these files, we pave the way for seamless access and utilization throughout the course of our project

In [ ]:
# # need to change the file directions for the new samples

# os.chdir('/content/drive/MyDrive/AD-Blocker Project')

# # Specify the path to your zip file
# zip_file_path = '/content/drive/MyDrive/AD-Blocker Project/new_ad_zip.zip'

# # Specify the directory where you want to extract the contents
# extract_dir = '/content/drive/MyDrive/AD-Blocker Project/data_one_second/ad_one_second'

# # Create the extraction directory if it doesn't exist
# os.makedirs(extract_dir, exist_ok=True)

# # Extract the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)

# ***Extracting Audio Embeddings***
Now that we have successfully loaded our audio data, the next step is to extract meaningful embeddings from it. This function - **"extract_audio_embedding"** employs OpenL3 to extract audio embeddings from the input audio data. Let's explore how it's done.

In [4]:
def extract_audio_embedding(audio_data, sample_rate):
    """
    Extract audio embeddings using OpenL3.

    Parameters:
    - audio_data: The audio signal.
    - sample_rate: The sample rate of the audio signal.

    Returns:
    - flat_embedding: The flattened audio embedding.
    """

    # Set the desired embedding size
    embedding_size = 512

    try:
        # Extract embeddings using OpenL3
        embedding, _ = openl3.get_audio_embedding(audio_data, sample_rate,
                                                  content_type="music",
                                                  embedding_size=embedding_size)

        # Flatten the embedding
        flat_embedding = np.ravel(embedding)

        return flat_embedding

    except Exception as e:
        # Handle any errors that might occur during the embedding
        # extraction process
        print("Error occurred during audio embedding extraction:", e)
        return None

# **Loading Audio Files**
Before processing any audio data, it's crucial to load the audio files correctly. This function, load_audio, accomplishes precisely that. It takes a file path as input and returns the audio signal along with its sample rate. Let's delve into the details of this function.

In [5]:
def load_audio(file_path):
    """
    Load an audio file and return the audio data and sample rate.

    Parameters:
    - file_path: The path to the audio file.

    Returns:
    - audio_data: The audio signal.
    - sample_rate: The sample rate of the audio file.
    """
    try:
        # Load the audio file using librosa
        audio_data, sample_rate = librosa.load(file_path, sr=None)

        return audio_data, sample_rate
    except Exception as e:
        # Handle any errors that might occur during the loading process
        print("Error occurred during audio loading:", e)
        return None, None

Loading Audio Data

In [6]:
import os
import librosa
import time

def load_data(data_folder, batch_size=4):  #TODO 4
    """
    Extract audio embeddings from audio files in a specific folder.

    Parameters:
    - data_folder: The path to the folder containing audio files.
    - batch_size: Number of audio files to process simultaneously.

    Returns:
    - embeddings: List of audio embeddings.
    """
    embeddings = []
    batch = []

    for file_name in os.listdir(data_folder):
        file_path = os.path.join(data_folder, file_name)
        if file_name.endswith('.wav'):  # Process only WAV audio files
            duration = librosa.get_duration(filename=file_path)
            if duration < 3.0:  # Check duration  #TODO
                continue
            audio_data, sample_rate = load_audio(file_path)  # Define load_audio function
            batch.append((audio_data, sample_rate, file_name))
            if len(batch) == batch_size:
                embeddings.extend(process_batch(batch))
                batch = []

    # Process the remaining files if the batch size doesn't divide evenly into the total number of files
    if batch:
        embeddings.extend(process_batch(batch))

    return embeddings

def process_batch(batch):
    """
    Process a batch of audio files and extract embeddings.

    Parameters:
    - batch: List of tuples, each containing audio data, sample rate, and file name.

    Returns:
    - batch_embeddings: List of audio embeddings for the batch.
    """
    batch_embeddings = []

    for audio_data, sample_rate, file_name in batch:
        start_time = time.time()
        embedding = extract_audio_embedding(audio_data, sample_rate)  # Define extract_audio_embedding function
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Execution time for {file_name}: {execution_time} seconds")
        batch_embeddings.append(embedding)

    return batch_embeddings

# You need to define the load_audio and extract_audio_embedding functions separately.


In [ ]:
# import os
# import librosa
# import time

# def load_data(data_folder):
#     """
#     Extract audio embeddings from audio files in a specific folder.

#     Parameters:
#     - data_folder: The path to the folder containing audio files.

#     Returns:
#     - embeddings: List of audio embeddings.
#     """
#     embeddings = []

#     for file_name in os.listdir(data_folder):
#         file_path = os.path.join(data_folder, file_name)
#         if file_name.endswith('.wav'):  # Process only WAV audio files
#             duration = librosa.get_duration(filename=file_path)
#             if duration < 5.0:  # Check duration
#                 continue
#             audio_data, sample_rate = load_audio(file_path)  # Define load_audio function
#             start_time = time.time()
#             embedding = extract_audio_embedding(audio_data, sample_rate)  # Define extract_audio_embedding function
#             end_time = time.time()
#             execution_time = end_time-start_time
#             print(f"Execution time: {execution_time} seconds" )
#             embeddings.append(embedding)

#     return embeddings




former audio data function for reference

Path to the audio samples of one second each

In [7]:
# # Path to data folders containing audio files in Google Drive
# pod_train_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_five_second/pod_five_second/train"
# ad_train_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_five_second/ad_five_second/train"
# pod_val_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_five_second/pod_five_second/validation"
# ad_val_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_five_second/ad_five_second/validation"
# pod_test_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_five_second/pod_five_second/test"
# ad_test_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_five_second/ad_five_second/test"


# # Directory paths for train, validation, and test sets in Google Drive 1 SEC
# pod_train_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_one_second/pod_one_second/train"
# ad_train_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_one_second/ad_one_second/train"
# pod_val_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_one_second/pod_one_second/validation"
# ad_val_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_one_second/ad_one_second/validation"
# pod_test_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_one_second/pod_one_second/test"
# ad_test_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_one_second/ad_one_second/test"

# Directory paths for train, validation, and test sets in Google Drive 3 SEC
# pod_train_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_three_second/pod_three_second/train"
# ad_train_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_three_second/ad_three_second/train"
# pod_val_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_three_second/pod_three_second/validation"
# ad_val_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_three_second/ad_three_second/validation"
# pod_test_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_three_second/pod_three_second/test"
# ad_test_data_folder = "/content/drive/MyDrive/AD-Blocker Project/data_three_second/ad_three_second/test"


# Directory paths for live radio folder with 3 sec samples
live_radio_folder = "/content/drive/MyDrive/AD-Blocker Project/real_time_audio/live_audio_3_sec"

In [8]:
# process live radio 3 sec samepls
live_radio_embeddings = load_data(live_radio_folder)
live_radio_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_live_radio_3_sec/OpenL3_live_radio_3_sec_embeddings.npy"
np.save(live_radio_embeddings_path, live_radio_embeddings)

<ipython-input-6-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)
<ipython-input-6-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 9s 9s/step
Execution time for live_radio_recording_segment_10.wav: 10.836166620254517 seconds
1/1 [==============================] - 0s 484ms/step
Execution time for live_radio_recording_segment_52.wav: 1.6052486896514893 seconds
1/1 [==============================] - 0s 412ms/step
Execution time for live_radio_recording_segment_75.wav: 1.9321479797363281 seconds
1/1 [==============================] - 0s 322ms/step
Execution time for live_radio_recording_segment_5.wav: 1.080928087234497 seconds


<ipython-input-6-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 1s 584ms/step
Execution time for live_radio_recording_segment_32.wav: 1.8849711418151855 seconds


1/1 [==============================] - 1s 566ms/step
Execution time for live_radio_recording_segment_7.wav: 1.9856195449829102 seconds
1/1 [==============================] - 3s 3s/step
Execution time for live_radio_recording_segment_57.wav: 4.19879150390625 seconds
1/1 [==============================] - 1s 699ms/step
Execution time for live_radio_recording_segment_62.wav: 3.778538703918457 seconds


...

1/1 [==============================] - 0s 302ms/step
Execution time for live_radio_recording_segment_707.wav: 1.0514578819274902 seconds
1/1 [==============================] - 0s 297ms/step
Execution time for live_radio_recording_segment_704.wav: 1.0319209098815918 seconds
1/1 [==============================] - 0s 303ms/step
Execution time for live_radio_recording_segment_702.wav: 1.0881295204162598 seconds
1/1 [==============================] - 0s 319ms/step
Execution time for live_radio_recording_segment_698.wav: 1.0749125480651855 seconds


<ipython-input-6-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 1s 581ms/step
Execution time for live_radio_recording_segment_694.wav: 2.3283536434173584 seconds
1/1 [==============================] - 1s 710ms/step
Execution time for live_radio_recording_segment_697.wav: 1.829819917678833 seconds
1/1 [==============================] - 1s 565ms/step
Execution time for live_radio_recording_segment_693.wav: 1.6379575729370117 seconds
1/1 [==============================] - 1s 601ms/step
Execution time for live_radio_recording_segment_699.wav: 1.7550432682037354 seconds


<ipython-input-6-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 368ms/step
Execution time for live_radio_recording_segment_689.wav: 1.1653704643249512 seconds


Loading Training, Validation, and Test Data

In [ ]:
# pod_train_embeddings = load_data(pod_train_data_folder)
# pod_train_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_train_embeddings.npy"
# np.save(pod_train_embeddings_path, pod_train_embeddings)

In [ ]:
ad_train_embeddings = load_data(ad_train_data_folder)
ad_train_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_ad_train_embeddings.npy"
np.save(ad_train_embeddings_path, ad_train_embeddings)

<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)
<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 9s 9s/step
Execution time for Yahoo Golfe Radio Commercial_segment_1.wav: 10.094384670257568 seconds
1/1 [==============================] - 0s 297ms/step
Execution time for KFC Radio Spot_segment_2.wav: 0.9159009456634521 seconds
1/1 [==============================] - 0s 294ms/step
Execution time for KFC Radio Spot_segment_16.wav: 0.9133589267730713 seconds
1/1 [==============================] - 0s 296ms/step
Execution time for Uber Radio Ad_segment_6.wav: 0.9842991828918457 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 390ms/step
Execution time for KFC Radio Spot_segment_5.wav: 1.1870861053466797 seconds


1/1 [==============================] - 0s 485ms/step
Execution time for Honda Radio Spot_segment_11.wav: 1.3192331790924072 seconds
1/1 [==============================] - 0s 488ms/step
Execution time for Yahoo Golfe Radio Commercial_segment_9.wav: 1.718639850616455 seconds
1/1 [==============================] - 0s 328ms/step
Execution time for Pride Powerwash Washing Machine Detergent New Radio Commercial 2021 OCQs Version_segment_10.wav: 0.9778311252593994 seconds


...

1/1 [==============================] - 0s 453ms/step
Execution time for Nike Find Your Greatness_segment_110.wav: 1.187958002090454 seconds
1/1 [==============================] - 1s 534ms/step
Execution time for LIPTON GREEN LEAF TEA Mmm Radio_segment_15.wav: 1.289003849029541 seconds
1/1 [==============================] - 1s 541ms/step
Execution time for Nike Find Your Greatness_segment_115.wav: 1.5056853294372559 seconds
1/1 [==============================] - 0s 498ms/step
Execution time for LIPTON GREEN LEAF TEA Mmm Radio_segment_2.wav: 1.513434648513794 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 348ms/step
Execution time for Nike Find Your Greatness_segment_91.wav: 1.0159461498260498 seconds
1/1 [==============================] - 0s 325ms/step
Execution time for Planet Fitness Radio Spot_segment_3.wav: 1.0244598388671875 seconds
1/1 [==============================] - 0s 337ms/step
Execution time for Coke Zero Radio Ad_segment_5.wav: 1.041569709777832 seconds
1/1 [==============================] - 0s 328ms/step
Execution time for AT and T Roleplay Radio Commercial_segment_4.wav: 0.9760630130767822 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 324ms/step
Execution time for Honda Radio Ad March 2015_segment_4.wav: 0.9900670051574707 seconds
1/1 [==============================] - 0s 345ms/step
Execution time for Eds Heinz Ad_segment_4.wav: 0.9923491477966309 seconds
1/1 [==============================] - 0s 316ms/step
Execution time for Jack In The Box Voice Over by DC Douglas_segment_10.wav: 0.9666392803192139 seconds
1/1 [==============================] - 0s 307ms/step
Execution time for PlayStation 5  Play Like Never Before_segment_6.wav: 0.9677283763885498 seconds


In [ ]:
pod_val_embeddings = load_data(pod_val_data_folder)
pod_val_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_val_embeddings.npy"
np.save(pod_val_embeddings_path, pod_val_embeddings)

<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 333ms/step
Execution time for TheDocumentaryPodcast-20231221-AssignmentUkraineBuildingBackBetter_segment_316.wav: 1.4769847393035889 seconds
1/1 [==============================] - 0s 313ms/step
Execution time for TheDocumentaryPodcast-20231221-AssignmentUkraineBuildingBackBetter_segment_204.wav: 1.3349723815917969 seconds
1/1 [==============================] - 0s 316ms/step
Execution time for TheDocumentaryPodcast-20231221-AssignmentUkraineBuildingBackBetter_segment_103.wav: 1.0957074165344238 seconds
1/1 [==============================] - 1s 514ms/step
Execution time for TheDocumentaryPodcast-20231221-AssignmentUkraineBuildingBackBetter_segment_195.wav: 1.3599450588226318 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 1s 553ms/step
Execution time for TheDocumentaryPodcast-20231221-AssignmentUkraineBuildingBackBetter_segment_257.wav: 2.179766893386841 seconds
1/1 [==============================] - 1s 605ms/step
Execution time for TheDocumentaryPodcast-20231221-AssignmentUkraineBuildingBackBetter_segment_145.wav: 1.9919190406799316 seconds
1/1 [==============================] - 1s 1s/step
Execution time for TheDocumentaryPodcast-20231221-AssignmentUkraineBuildingBackBetter_segment_137.wav: 5.853079319000244 seconds
1/1 [==============================] - 0s 485ms/step
Execution time for TheDocumentaryPodcast-20231221-AssignmentUkraineBuildingBackBetter_segment_112.wav: 2.48213529586792 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


...

1/1 [==============================] - 0s 474ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_228.wav: 1.552567958831787 seconds
1/1 [==============================] - 0s 301ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_211.wav: 1.271688461303711 seconds
1/1 [==============================] - 0s 298ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_215.wav: 1.0539109706878662 seconds
1/1 [==============================] - 0s 321ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_232.wav: 1.0961365699768066 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 318ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_212.wav: 1.1410727500915527 seconds
1/1 [==============================] - 0s 299ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_242.wav: 1.1183698177337646 seconds
1/1 [==============================] - 0s 329ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_224.wav: 1.107374906539917 seconds
1/1 [==============================] - 0s 307ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_234.wav: 1.110886573791504 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 301ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_221.wav: 1.134253978729248 seconds
1/1 [==============================] - 0s 308ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_239.wav: 1.1047165393829346 seconds
1/1 [==============================] - 0s 446ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_233.wav: 1.435265064239502 seconds


In [ ]:
ad_val_embeddings = load_data(ad_val_data_folder)
ad_val_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_ad_val_embeddings.npy"
np.save(ad_val_embeddings_path, ad_val_embeddings)


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 409ms/step
Execution time for Ikea Get a Kitchen Installed Radio - AdNews_segment_1.wav: 1.3881442546844482 seconds
1/1 [==============================] - 1s 1s/step
Execution time for PopCorners Breaking Bad Super Bowl Commercial Breaking Good 60_segment_7.wav: 1.9390437602996826 seconds
1/1 [==============================] - 0s 450ms/step
Execution time for Wolt Stealing Time Sakiot_segment_11.wav: 1.1231930255889893 seconds
1/1 [==============================] - 0s 479ms/step
Execution time for PopCorners Breaking Bad Super Bowl Commercial Breaking Good 60_segment_11.wav: 1.2454736232757568 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 395ms/step
Execution time for Wolt Stealing Time Sakiot_segment_10.wav: 1.2687633037567139 seconds
1/1 [==============================] - 0s 422ms/step
Execution time for Abel & Cole 2023 radio ad Difficult tastes better_segment_2.wav: 1.1333997249603271 seconds
1/1 [==============================] - 0s 366ms/step
Execution time for Ikea Get a Kitchen Installed Radio - AdNews_segment_4.wav: 1.0493898391723633 seconds
1/1 [==============================] - 0s 341ms/step
Execution time for Verizon Wireless Radio Spot Inauguration_segment_18.wav: 1.0465645790100098 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


...

1/1 [==============================] - 0s 389ms/step
Execution time for McDonalds radio ad for Filet-O-fish_segment_7.wav: 1.1439330577850342 seconds
1/1 [==============================] - 0s 347ms/step
Execution time for PopCorners Breaking Bad Super Bowl Commercial Breaking Good 60_segment_9.wav: 1.042191982269287 seconds
1/1 [==============================] - 0s 326ms/step
Execution time for PopCorners Breaking Bad Super Bowl Commercial Breaking Good 60_segment_1.wav: 1.0532302856445312 seconds
1/1 [==============================] - 0s 482ms/step
Execution time for Christmas Radio Ad_segment_3.wav: 1.1614866256713867 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 321ms/step
Execution time for McDonalds radio ad for Filet-O-fish_segment_9.wav: 1.0122997760772705 seconds
1/1 [==============================] - 0s 315ms/step
Execution time for McDonalds radio ad for Filet-O-fish_segment_5.wav: 0.9550490379333496 seconds
1/1 [==============================] - 0s 309ms/step
Execution time for Abel & Cole 2023 radio ad Difficult tastes better_segment_8.wav: 0.9968750476837158 seconds
1/1 [==============================] - 0s 363ms/step
Execution time for Wolt Stealing Time Sakiot_segment_4.wav: 0.9935169219970703 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 315ms/step
Execution time for Pizza Hut Radio - World Cup English_segment_3.wav: 0.9533207416534424 seconds


In [ ]:
pod_test_embeddings = load_data(pod_test_data_folder)
pod_test_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_test_embeddings.npy"
np.save(pod_test_embeddings_path, pod_test_embeddings)


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 326ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_398.wav: 1.1454098224639893 seconds
1/1 [==============================] - 0s 300ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_27.wav: 1.1208064556121826 seconds
1/1 [==============================] - 0s 301ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_397.wav: 1.103449821472168 seconds
1/1 [==============================] - 0s 324ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_346.wav: 1.0910015106201172 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 1s 522ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_61.wav: 1.960988998413086 seconds
1/1 [==============================] - 2s 2s/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_343.wav: 3.308668613433838 seconds
1/1 [==============================] - 1s 829ms/step
Execution time for TheDocumentaryPodcast-20231230-BBCOSConversationAdventurers_segment_275.wav: 4.198242902755737 seconds
1/1 [==============================] - 0s 477ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_88.wav: 2.2218148708343506 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


...

1/1 [==============================] - 0s 333ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_522.wav: 1.2796506881713867 seconds
1/1 [==============================] - 0s 454ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_545.wav: 1.5216436386108398 seconds
1/1 [==============================] - 0s 407ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_531.wav: 1.5332508087158203 seconds
1/1 [==============================] - 0s 305ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_528.wav: 1.075298547744751 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 313ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_539.wav: 1.0537023544311523 seconds
1/1 [==============================] - 0s 312ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_532.wav: 1.0607004165649414 seconds
1/1 [==============================] - 0s 309ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_533.wav: 1.0977599620819092 seconds
1/1 [==============================] - 0s 314ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_526.wav: 1.081451416015625 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 314ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_525.wav: 1.088630199432373 seconds
1/1 [==============================] - 0s 302ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_549.wav: 1.099386215209961 seconds
1/1 [==============================] - 0s 318ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_552.wav: 1.0853862762451172 seconds
1/1 [==============================] - 0s 462ms/step
Execution time for TheDocumentaryPodcast-20240110-SolutionsJournalismBuildingAFutureForCycloneHitMozambique_segment_536.wav: 1.4016528129577637 seconds


In [ ]:
ad_test_embeddings = load_data(ad_test_data_folder)
ad_test_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_ad_test_embeddings.npy"
np.save(ad_test_embeddings_path, ad_test_embeddings)

<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 1s 552ms/step
Execution time for McDonalds Commercial 2022 - USA_segment_2.wav: 1.462491750717163 seconds
1/1 [==============================] - 0s 395ms/step
Execution time for Lays Super Bowl 2022 with  Seth Rogen and Paul Rudd_segment_20.wav: 1.5485260486602783 seconds
1/1 [==============================] - 0s 305ms/step
Execution time for The Travis Scott Meal - McDonalds Official Commercial_segment_9.wav: 0.9425296783447266 seconds
1/1 [==============================] - 0s 305ms/step
Execution time for Burger King Radio_segment_15.wav: 0.9652252197265625 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 1s 1s/step
Execution time for The Travis Scott Meal - McDonalds Official Commercial_segment_10.wav: 1.7081689834594727 seconds
1/1 [==============================] - 0s 458ms/step
Execution time for Southwest Air Job Interview Radio Commercial_segment_6.wav: 1.1441009044647217 seconds
1/1 [==============================] - 0s 399ms/step
Execution time for Lays Super Bowl 2022 with  Seth Rogen and Paul Rudd_segment_6.wav: 1.0815765857696533 seconds
1/1 [==============================] - 0s 357ms/step
Execution time for Burger King Radio_segment_13.wav: 1.0233705043792725 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


...

1/1 [==============================] - 0s 380ms/step
Execution time for The Travis Scott Meal - McDonalds Official Commercial_segment_3.wav: 1.030916690826416 seconds
1/1 [==============================] - 0s 331ms/step
Execution time for Lowes Radio Spot_segment_9.wav: 1.00946044921875 seconds
1/1 [==============================] - 0s 333ms/step
Execution time for McDonalds Commercial 2022 - USA_segment_5.wav: 1.0170071125030518 seconds
1/1 [==============================] - 0s 352ms/step
Execution time for Lowes Radio Spot_segment_15.wav: 1.0248334407806396 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 320ms/step
Execution time for Lays Super Bowl 2022 with  Seth Rogen and Paul Rudd_segment_9.wav: 1.0988209247589111 seconds
1/1 [==============================] - 0s 318ms/step
Execution time for McDonalds Commercial 2022 - USA_segment_6.wav: 0.9351556301116943 seconds
1/1 [==============================] - 0s 453ms/step
Execution time for Burger King Radio_segment_14.wav: 1.2257943153381348 seconds
1/1 [==============================] - 0s 444ms/step
Execution time for Mr. Clean Radio Commercial 2023_segment_7.wav: 1.264134407043457 seconds


<ipython-input-8-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 310ms/step
Execution time for McDonalds Commercial 2022 - USA_segment_3.wav: 1.1073896884918213 seconds
1/1 [==============================] - 0s 306ms/step
Execution time for Burger King Radio_segment_7.wav: 0.9282505512237549 seconds
1/1 [==============================] - 0s 311ms/step
Execution time for Southwest Air Job Interview Radio Commercial_segment_2.wav: 0.9758338928222656 seconds


In [ ]:
pod_train_data_folder_1 = "/content/drive/MyDrive/AD-Blocker Project/data_three_second/pod_three_second/train/train_part1"
pod_train_data_folder_2 = "/content/drive/MyDrive/AD-Blocker Project/data_three_second/pod_three_second/train/train_part2"
pod_train_data_folder_3 = "/content/drive/MyDrive/AD-Blocker Project/data_three_second/pod_three_second/train/train_part3"
pod_train_data_folder_4 = "/content/drive/MyDrive/AD-Blocker Project/data_three_second/pod_three_second/train/train_part4"

In [ ]:
pod_train_embeddings = load_data(pod_train_data_folder_1)
pod_train_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_train_embeddings_1.npy"
np.save(pod_train_embeddings_path, pod_train_embeddings)

<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)
<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 9s 9s/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_472.wav: 11.560227394104004 seconds
1/1 [==============================] - 0s 295ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_314.wav: 1.0149102210998535 seconds
1/1 [==============================] - 0s 445ms/step
Execution time for TheDocumentaryPodcast-20240113-BBCOSConversationsOnGraduateUnemployment_segment_74.wav: 1.1850647926330566 seconds
1/1 [==============================] - 0s 432ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_415.wav: 1.372715950012207 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 292ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_366.wav: 1.5031118392944336 seconds


1/1 [==============================] - 0s 311ms/step
Execution time for TheDocumentaryPodcast-20240113-BBCOSConversationsOnGraduateUnemployment_segment_73.wav: 1.036116361618042 seconds
1/1 [==============================] - 0s 305ms/step
Execution time for TheDocumentaryPodcast-20240113-BBCOSConversationsOnGraduateUnemployment_segment_54.wav: 1.0130226612091064 seconds
1/1 [==============================] - 0s 328ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_364.wav: 1.0417029857635498 seconds


...

1/1 [==============================] - 0s 452ms/step
Execution time for TheDocumentaryPodcast-20240113-BBCOSConversationsOnGraduateUnemployment_segment_138.wav: 1.213700294494629 seconds
1/1 [==============================] - 0s 377ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_352.wav: 1.157498836517334 seconds
1/1 [==============================] - 0s 363ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_451.wav: 1.1556189060211182 seconds
1/1 [==============================] - 0s 328ms/step
Execution time for TheDocumentaryPodcast-20240113-BBCOSConversationsOnGraduateUnemployment_segment_47.wav: 1.0914483070373535 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 315ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_503.wav: 1.1000609397888184 seconds
1/1 [==============================] - 0s 312ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_456.wav: 1.0609521865844727 seconds
1/1 [==============================] - 0s 305ms/step
Execution time for TheDocumentaryPodcast-20240113-BBCOSConversationsOnGraduateUnemployment_segment_39.wav: 1.0840363502502441 seconds
1/1 [==============================] - 0s 461ms/step
Execution time for TheDocumentaryPodcast-20240113-BBCOSConversationsOnGraduateUnemployment_segment_86.wav: 1.4010660648345947 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 453ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_403.wav: 1.4128077030181885 seconds
1/1 [==============================] - 0s 302ms/step
Execution time for TheDocumentaryPodcast-20240113-BBCOSConversationsOnGraduateUnemployment_segment_103.wav: 1.2454383373260498 seconds
1/1 [==============================] - 0s 329ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_295.wav: 1.0972959995269775 seconds
1/1 [==============================] - 0s 322ms/step
Execution time for TheDocumentaryPodcast-20231224-FootballAndFaith_segment_405.wav: 1.042463779449463 seconds


In [ ]:
pod_train_embeddings = load_data(pod_train_data_folder_2)
pod_train_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_train_embeddings_2.npy"
np.save(pod_train_embeddings_path, pod_train_embeddings)

<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 307ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_481.wav: 1.0945627689361572 seconds
1/1 [==============================] - 0s 295ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_424.wav: 1.0685243606567383 seconds
1/1 [==============================] - 0s 320ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_298.wav: 1.068943977355957 seconds
1/1 [==============================] - 0s 298ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_223.wav: 1.0224504470825195 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 1s 512ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_346.wav: 1.291358470916748 seconds
1/1 [==============================] - 0s 294ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_144.wav: 1.154416561126709 seconds
1/1 [==============================] - 0s 308ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_96.wav: 1.0634429454803467 seconds
1/1 [==============================] - 0s 313ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_301.wav: 1.0681190490722656 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


...

1/1 [==============================] - 0s 301ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_125.wav: 1.062628984451294 seconds
1/1 [==============================] - 0s 293ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_216.wav: 1.0435900688171387 seconds
1/1 [==============================] - 0s 290ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_257.wav: 1.04420804977417 seconds
1/1 [==============================] - 0s 289ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_319.wav: 1.0216281414031982 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 420ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_17.wav: 1.2996132373809814 seconds
1/1 [==============================] - 0s 428ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_484.wav: 1.4123780727386475 seconds
1/1 [==============================] - 0s 289ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_338.wav: 1.3369662761688232 seconds
1/1 [==============================] - 0s 294ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_22.wav: 1.047182559967041 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 307ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_181.wav: 1.0296204090118408 seconds
1/1 [==============================] - 0s 302ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_275.wav: 1.01753830909729 seconds
1/1 [==============================] - 0s 303ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_68.wav: 1.029081106185913 seconds
1/1 [==============================] - 0s 307ms/step
Execution time for GlobalNewsPodcast-20240409-NorthAmericaAwedByTotalSolarEclipse_segment_335.wav: 1.0214934349060059 seconds


In [ ]:
pod_train_embeddings = load_data(pod_train_data_folder_3)
pod_train_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_train_embeddings_3.npy"
np.save(pod_train_embeddings_path, pod_train_embeddings)

<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 292ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_373.wav: 1.0544347763061523 seconds
1/1 [==============================] - 0s 431ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_329.wav: 1.4227077960968018 seconds
1/1 [==============================] - 1s 1s/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_13.wav: 2.488729476928711 seconds
1/1 [==============================] - 0s 420ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_160.wav: 1.215742826461792 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 427ms/step
Execution time for TheDocumentaryPodcast-20231216-BBCOSConversationsUkrainiansAndHope_segment_419.wav: 1.244002342224121 seconds
1/1 [==============================] - 0s 418ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_67.wav: 1.2211151123046875 seconds
1/1 [==============================] - 0s 390ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_6.wav: 1.1967973709106445 seconds
1/1 [==============================] - 0s 332ms/step
Execution time for TheDocumentaryPodcast-20231216-BBCOSConversationsUkrainiansAndHope_segment_428.wav: 1.117849588394165 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


...

1/1 [==============================] - 0s 315ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_101.wav: 1.0494627952575684 seconds
1/1 [==============================] - 0s 320ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_150.wav: 1.0914576053619385 seconds
1/1 [==============================] - 0s 323ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_119.wav: 1.0565640926361084 seconds
1/1 [==============================] - 0s 308ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_68.wav: 1.058206558227539 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 308ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_47.wav: 1.0193777084350586 seconds
1/1 [==============================] - 0s 300ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_196.wav: 1.0264883041381836 seconds
1/1 [==============================] - 0s 455ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_168.wav: 1.203805685043335 seconds
1/1 [==============================] - 0s 419ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_126.wav: 1.3489813804626465 seconds


<ipython-input-7-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 319ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_253.wav: 1.3792130947113037 seconds
1/1 [==============================] - 0s 290ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_377.wav: 1.00734281539917 seconds
1/1 [==============================] - 0s 393ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_348.wav: 1.123307228088379 seconds
1/1 [==============================] - 0s 431ms/step
Execution time for TheDocumentaryPodcast-20240106-BBCOSConversationsCovid19FourYearsLater_segment_155.wav: 1.4254684448242188 seconds


In [ ]:
pod_train_embeddings = load_data(pod_train_data_folder_4)
pod_train_embeddings_path = "/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_train_embeddings_4.npy"
np.save(pod_train_embeddings_path, pod_train_embeddings)

<ipython-input-6-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)
<ipython-input-6-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 9s 9s/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_116.wav: 11.201151847839355 seconds
1/1 [==============================] - 0s 311ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_136.wav: 1.0989885330200195 seconds
1/1 [==============================] - 0s 291ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_307.wav: 1.082319974899292 seconds
1/1 [==============================] - 0s 297ms/step
Execution time for TheDocumentaryPodcast-20240116-OurHouseStoriesOfTheHolocaust_segment_548.wav: 1.0656507015228271 seconds


<ipython-input-6-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 304ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_192.wav: 1.03519606590271 seconds


1/1 [==============================] - 0s 338ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_9.wav: 1.0716235637664795 seconds
1/1 [==============================] - 0s 446ms/step
Execution time for TheDocumentaryPodcast-20240116-OurHouseStoriesOfTheHolocaust_segment_537.wav: 1.2346487045288086 seconds
1/1 [==============================] - 0s 414ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_237.wav: 1.426952838897705 seconds


...

1/1 [==============================] - 0s 439ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_3.wav: 1.2000250816345215 seconds
1/1 [==============================] - 0s 417ms/step
Execution time for TheDocumentaryPodcast-20240116-OurHouseStoriesOfTheHolocaust_segment_447.wav: 1.4591073989868164 seconds
1/1 [==============================] - 0s 439ms/step
Execution time for TheDocumentaryPodcast-20240116-OurHouseStoriesOfTheHolocaust_segment_428.wav: 1.4612541198730469 seconds
1/1 [==============================] - 0s 291ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_259.wav: 1.1393709182739258 seconds


<ipython-input-6-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 309ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_137.wav: 1.0416920185089111 seconds
1/1 [==============================] - 0s 300ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_124.wav: 1.075284481048584 seconds
1/1 [==============================] - 0s 309ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_25.wav: 1.029083013534546 seconds
1/1 [==============================] - 0s 298ms/step
Execution time for TheDocumentaryPodcast-20240116-OurHouseStoriesOfTheHolocaust_segment_543.wav: 1.049863338470459 seconds


<ipython-input-6-8827571c53da>:22: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


1/1 [==============================] - 0s 311ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_27.wav: 1.043605089187622 seconds
1/1 [==============================] - 0s 290ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_4.wav: 1.0268759727478027 seconds
1/1 [==============================] - 0s 435ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_233.wav: 1.698387622833252 seconds
1/1 [==============================] - 1s 507ms/step
Execution time for TheDocumentaryPodcast-20240413-BBCOSConversationsSudansWarOneYearOn_segment_359.wav: 1.2514019012451172 seconds


In [ ]:
# import joblib

# # Save the trained model to a file
# joblib_file = "openL3_model.pkl"
# joblib.dump(openL3_model, joblib_file)


In [ ]:
# from google.colab import files

# # Download the file
# files.download(joblib_file)
